## Import necessary packages

In [2]:
import pandas as pd
import numpy as np
import scipy.integrate as integ
import matplotlib.pyplot as plt

def figure_size_setting(WIDTH):
    #WIDTH = 700.0  # the number latex spits out
    FACTOR = 0.8  # the fraction of the width you'd like the figure to occupy
    fig_width_pt  = WIDTH * FACTOR
    inches_per_pt = 1.0 / 72.27
    golden_ratio  = (np.sqrt(5) - 1.0) / 2.0  # because it looks good
    fig_width_in  = fig_width_pt * inches_per_pt  # figure width in inches
    fig_height_in = fig_width_in * golden_ratio   # figure height in inches
    fig_dims    = [fig_width_in, fig_height_in] # fig dims as a list
    return fig_dims

SMALL_SIZE = 12
MEDIUM_SIZE = 15
BIGGER_SIZE = 15
plt.rc('font', size=SMALL_SIZE, family='sans-serif', serif='Arial')          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('text')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Arial'] + plt.rcParams['font.serif']
plt.rcParams['mathtext.default'] = 'regular'

## Sample species with random abilities to form a species pool for the community assembly

In [3]:
#### draw a random configuratoin
from scipy.stats import bernoulli
Nr = 40
Ne = 20
Nb = 10000
m2b_full_before_evolution = np.random.rand(Nr,Nb)
m2b_full_before_evolution[:int(Nr/2),:] = np.random.rand(int(Nr/2),Nb) * (np.random.rand(int(Nr/2),Nb) < 0.7).astype(int)
m2b_full_before_evolution[int(Nr/2):,:] = np.random.rand(int(Nr/2),Nb) * (np.random.rand(int(Nr/2),Nb) < 0.2).astype(int)

met_ess_full_before_evolution = np.ones((Ne,Nb))
met_ess_full_before_evolution = met_ess_full_before_evolution * (np.random.rand(Ne,Nb) < 0.7).astype(int)


In [4]:
print(m2b_full_before_evolution.shape)
print(met_ess_full_before_evolution.shape)


(40, 10000)
(20, 10000)


In [5]:
def CRM_microbial_community(i_microbe_subset, consumption_mat, yield_mat, depletion_mat, met_ess):
    delta = 1/24 # dilution rate
    
    Ne = met_ess_full.shape[0]
    multiplier_array = (1 - 1/Ne)**(met_ess==0).astype(int).sum(0)
    #multiplier_array = np.ones((met_ess_full.shape[1]))
    multiplier_array = multiplier_array[i_microbe_subset]
    consumption_mat = consumption_mat[i_microbe_subset, :] * multiplier_array[:, np.newaxis]
    yield_mat = yield_mat[i_microbe_subset, :]
    depletion_mat = depletion_mat[:, i_microbe_subset] * multiplier_array[np.newaxis, :]

    Db = len(i_microbe_subset)
    Dr = consumption_mat.shape[1]

    ts = np.linspace(0, 500000, 1000)
    B0 = np.array([1] * Db)
    R0 = np.array([1] * Dr)
    Db = len(B0)
    Dr = len(R0)
    y0 = np.concatenate([B0, R0])
    
    def dydt(t, y):
        R_supply = np.zeros([Dr])
        R_supply[:] = 1

        dydt = np.zeros([Db+Dr])
        B = y[:Db]
        R = y[Db:(Db+Dr)]
        
        dBdt = B * np.dot(consumption_mat, R) - delta * B
        dRdt = delta * R_supply - R * np.dot(depletion_mat, B) - delta * R   
        dydt[:Db] = dBdt
        dydt[Db:(Db+Dr)] = dRdt

        return dydt

    t_min=0
    t_max=1e6
    solution=integ.LSODA(dydt,t_min,y0,t_max)
    solution=integ.solve_ivp(dydt,[t_min, t_max],y0,method='LSODA', atol=1e-4, rtol=1e-6)
    ts = solution.t
    Ps = solution.y.transpose()

    B = Ps[:,:Db]
    R = Ps[:,Db:(Db+Dr)]
    
    return Ps[:, :]


In [6]:
#### Subsample their genomic capabilities to decide the protein abilities

from scipy.stats import bernoulli
from numpy.matlib import repmat
m2b = m2b_full_before_evolution.copy()
for i in range(m2b.shape[1]):
    Connectance2 = np.random.uniform(low=0.0, high=1.0, size=1)[0]
    Nr, Nb = m2b_full_before_evolution.shape
    data_bern = bernoulli.rvs(size=Nr*Nb,p=Connectance2).reshape(Nr, Nb)
    m2b[:,i] = m2b_full_before_evolution[:,i] * data_bern[:,i]
m2b_full = m2b_full_before_evolution.copy()

met_ess = met_ess_full_before_evolution.copy()
for i in range(m2b.shape[1]):
    Connectance2 = np.random.uniform(low=0.0, high=1.0, size=1)[0]
    Nr, Nb = met_ess_full_before_evolution.shape
    data_bern = bernoulli.rvs(size=Ne*Nb,p=Connectance2).reshape(Ne, Nb)
    met_ess[:,i] = met_ess_full_before_evolution[:,i] * data_bern[:,i]
met_ess_full = met_ess_full_before_evolution.copy()

m2b_binary = m2b.copy()
m2b_binary[m2b_binary > 0] = 1
denominator = m2b_binary.sum(0); denominator[denominator <= 0] = 1
m2b = m2b / denominator
print(m2b_binary.shape)
print(m2b_binary.sum()/m2b_binary.shape[0]/m2b_binary.shape[1])


#### Generate parameters for the ODE simulation
s = m2b_binary.sum()/m2b_binary.shape[0]/m2b_binary.shape[1]
Nb = m2b_binary.shape[1] 
Nr = m2b_binary.shape[0]
consumption_mat = m2b.transpose().copy()
yield_mat = consumption_mat.copy()
depletion_mat = consumption_mat.copy().transpose()
i_microbe_subset = range(consumption_mat.shape[0])


(40, 10000)
0.225845


In [7]:
######## "consumption_mat" is the truely expressed consumption functions,
######## "consumption_mat_red_genome" is the consumption functions encoded in genomes,
######## "met_ess_red" is the truely expressed essential functions, 
######## "met_ess_red_genome" is the essential functions encoded in genomes

s1, s2 = consumption_mat.shape
A = np.zeros([s1+s2, s1+s2])
A[:s1,s1:] = consumption_mat
A[s1:,:s1] = consumption_mat.transpose()

solution = CRM_microbial_community(i_microbe_subset, consumption_mat, yield_mat, depletion_mat, met_ess)[-1,:]
abundance = solution[:Nb]
concentration = solution[Nb:]
i_survivors = np.where((abundance > 1e-5))[0]
consumption_mat_red = consumption_mat[i_survivors, :]  
consumption_mat_red_genome = m2b_full.transpose()[i_survivors, :].copy()
abundance_survivors = abundance[i_survivors]     
s1, s2 = consumption_mat_red.shape
A = np.zeros([s1+s2, s1+s2])
A[:s1,s1:] = consumption_mat_red
A[s1:,:s1] = consumption_mat_red.transpose()

met_ess_red = met_ess.transpose()[i_survivors, :]
met_ess_red_genome = met_ess_full.transpose()[i_survivors, :].copy()

In [8]:
print(np.mean(met_ess[:, i_survivors]!=0))
print(np.mean(met_ess!=0))


0.6569444444444444
0.34444


In [16]:
print(np.mean(consumption_mat[i_survivors,:]!=0))
print(np.mean(consumption_mat!=0))


0.052083333333333336
0.225845


In [17]:
####### pickle all matrices which are used to generate the simulation
'''
#### save a pickle file
import pickle
pickle_out = open("model_simulation_results.pickle", "wb")
pickle.dump([i_survivors, abundance_survivors, m2b_full_before_evolution, m2b_full, consumption_mat, consumption_mat_red_genome, consumption_mat_red, met_ess_full_before_evolution, met_ess_full, met_ess, met_ess_red_genome, met_ess_red], pickle_out)
pickle_out.close()
'''
